In [2]:
from keras.preprocessing.image import ImageDataGenerator


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# merge image dimension = 400,1590

train_dir = "/content/drive/MyDrive/Colab Notebooks/test2/MFCC_techno&semi_split/train/"
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(350,420),color_mode="rgba",class_mode='categorical',batch_size=13)

validation_dir = "/content/drive/MyDrive/Colab Notebooks/test2/MFCC_techno&semi_split/test/"
vali_datagen = ImageDataGenerator()
vali_generator = vali_datagen.flow_from_directory(validation_dir,target_size=(350,420),color_mode='rgba',class_mode='categorical',batch_size=13)

Found 49 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [31]:
# Get the class indices dictionary
class_indices = train_generator.class_indices

# Print the class names and their corresponding integer indices
print(class_indices)        ## output {'intra': 0, 'memor': 1, 'vert': 2}

{'semi': 0, 'techno': 1}


In [32]:
# from keras import layers
# from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
#                           Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout)
# from keras.models import Model, load_model

# def GModel(input_shape = (500,700,4),classes=2):
  
#   X_input = Input(input_shape)

#   X = Conv2D(8,kernel_size=(3,3),strides=(1,1))(X_input)
#   X = BatchNormalization(axis=3)(X)
#   X = Activation('relu')(X)
#   X = MaxPooling2D((2,2))(X)
  
#   X = Conv2D(16,kernel_size=(3,3),strides = (1,1))(X)
#   X = BatchNormalization(axis=3)(X)
#   X = Activation('relu')(X)
#   X = MaxPooling2D((2,2))(X)
  
#   X = Conv2D(32,kernel_size=(3,3),strides = (1,1))(X)
#   X = BatchNormalization(axis=3)(X)
#   X = Activation('relu')(X)
#   X = MaxPooling2D((2,2))(X)

#   X = Conv2D(64,kernel_size=(3,3),strides=(1,1))(X)
#   X = BatchNormalization(axis=-1)(X)
#   X = Activation('relu')(X)
#   X = MaxPooling2D((2,2))(X)
  
#   X = Conv2D(128,kernel_size=(3,3),strides=(1,1))(X)
#   X = BatchNormalization(axis=-1)(X)
#   X = Activation('relu')(X)
#   X = MaxPooling2D((2,2))(X)

  
#   X = Flatten()(X)
  
#   dropout_layer = Dropout(rate=0.5)
#   Y = dropout_layer(X)

#   X = Dense(classes, activation='sigmoid', name=str(classes))(Y)

#   model = Model(inputs=X_input,outputs=X,name='GModel')

#   return model

In [33]:
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout)
from keras.models import Model, load_model

def GModel(input_shape = (350,420,4),classes=2):
  
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1))(X_input)
  # X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(16,kernel_size=(3,3),strides = (1,1))(X)
  # X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(3,3),strides = (1,1))(X)
  # X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  dropout_layer1 = Dropout(rate=0.2)
  X = dropout_layer1(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1))(X)
  # X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  dropout_layer2 = Dropout(rate=0.3)
  X = dropout_layer2(X)

  X = Conv2D(128,kernel_size=(3,3),strides=(1,1))(X)
  # X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)
  
  dropout_layer3 = Dropout(rate=0.5)
  Y = dropout_layer3(X)

  X = Dense(classes, activation='sigmoid', name=str(classes))(Y)

  model = Model(inputs=X_input,outputs=X,name='GModel')

  return model

In [34]:
import keras.backend as K
from keras.optimizers import Adam


def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
  
model = GModel(input_shape=(350,420,4),classes=2)
opt = Adam(learning_rate=0.001)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

model.fit_generator(train_generator,epochs=45,validation_data=vali_generator)

<ipython-input-34-dd8d5d61b352>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=45,validation_data=vali_generator)


Epoch 1/45
4/4 [==============================] - 8s 2s/step - loss: 101.3414 - accuracy: 0.6735 - get_f1: 0.6514 - val_loss: 31.8306 - val_accuracy: 0.5000 - val_get_f1: 0.5000
Epoch 2/45
4/4 [==============================] - 7s 2s/step - loss: 24.5675 - accuracy: 0.6122 - get_f1: 0.6502 - val_loss: 4.1636 - val_accuracy: 0.5000 - val_get_f1: 0.5000
Epoch 3/45
4/4 [==============================] - 6s 1s/step - loss: 4.6252 - accuracy: 0.4694 - get_f1: 0.4730 - val_loss: 2.3547 - val_accuracy: 0.5000 - val_get_f1: 0.5000
Epoch 4/45
4/4 [==============================] - 8s 2s/step - loss: 3.8183 - accuracy: 0.7347 - get_f1: 0.7069 - val_loss: 2.2920 - val_accuracy: 0.5000 - val_get_f1: 0.5000
Epoch 5/45
4/4 [==============================] - 6s 1s/step - loss: 1.5465 - accuracy: 0.7347 - get_f1: 0.7361 - val_loss: 0.7461 - val_accuracy: 0.5000 - val_get_f1: 0.5000
Epoch 6/45
4/4 [==============================] - 8s 2s/step - loss: 0.7291 - accuracy: 0.6122 - get_f1: 0.4709 - val_los

In [36]:
model.save("/content/drive/MyDrive/Colab Notebooks/test2/xxmfcc_sigmoid_semi$techno.h5")